# This is how we automatically aligned the entire EGOCOM dataset - no human aid necessary.

In [1]:
from __future__ import print_function, absolute_import, division, unicode_literals, with_statement # Python 2 compatibility

import os
import numpy as np
import subprocess

In [2]:
# Read in the data

data_loc = "/home/cgn/Downloads/egocom-aligned/"
write_loc = "/home/cgn/Downloads/egocom-aligned-final/"
inter_loc = "/home/cgn/Downloads/egocom-intermediate/"
alignments_fn = "/home/cgn/Downloads/egocom-aligned-final/ALIGNMENT_ALL.txt"
fn_dict = {}
for fn in sorted([v for v in os.listdir(data_loc) if v[-4:] == ".MP4"]):
    key = fn[9:23] + fn[32:37] if 'part' in fn else fn[9:21]
    fn_dict[key] = fn_dict[key] + [fn] if key in fn_dict else [fn]
    
with open(alignments_fn, mode='r') as f:
    alignments = [z.strip() for z in f.readlines()]

In [3]:
samplerate = 44100
for alignment in alignments:
    key, lst = alignment.split(" | ")
    # Get alignment array
    alignment = (np.array(eval(lst)) / samplerate).round(4)
    
    files = [data_loc + z for z in fn_dict[key]]
    inter_files = [inter_loc + z for z in fn_dict[key]]
    output_files = [write_loc + z for z in fn_dict[key]]
    
    # Get the duration of each file
    d = []
    for file in files:
        cmd = "ffprobe -i {f} -show_format | grep duration".format(f=file)
        d.append(subprocess.getoutput(cmd).split("=")[-1])
    duration = np.array(d, dtype=float)

    print("Duration of original videos:", duration)
    # Find the shortest duration (minus the length we cut from the start)
    min_duration = min(duration - alignment)
    # Use the shortest duration to find the final duration for all 
    # videos so that they are the same length.
    final_duration = min_duration + alignment
    
    for i in range(len(alignment)):
        cmd = "ffmpeg -ss {s} -t {t} -i {i} -c copy {o}".format( 
            s=alignment[i], 
            t=min_duration,
            i=files[i],
            o=output_files[i],
        )
        print(cmd)
        subprocess.getoutput(cmd)

Duration of original videos: [274.9   274.948 275.   ]
ffmpeg -ss 0.0027 -t 274.8973 -i /home/cgn/Downloads/egocom-aligned/vid_001__day_1__con_1__person_1_part1.MP4 -c copy /home/cgn/Downloads/egocom-aligned-final/vid_001__day_1__con_1__person_1_part1.MP4
ffmpeg -ss 0.0 -t 274.8973 -i /home/cgn/Downloads/egocom-aligned/vid_006__day_1__con_1__person_2_part1.MP4 -c copy /home/cgn/Downloads/egocom-aligned-final/vid_006__day_1__con_1__person_2_part1.MP4
ffmpeg -ss 0.0043 -t 274.8973 -i /home/cgn/Downloads/egocom-aligned/vid_011__day_1__con_1__person_3_part1.MP4 -c copy /home/cgn/Downloads/egocom-aligned-final/vid_011__day_1__con_1__person_3_part1.MP4
Duration of original videos: [294.4   294.522 294.5  ]
ffmpeg -ss 0.0092 -t 294.39079999999996 -i /home/cgn/Downloads/egocom-aligned/vid_002__day_1__con_1__person_1_part2.MP4 -c copy /home/cgn/Downloads/egocom-aligned-final/vid_002__day_1__con_1__person_1_part2.MP4
ffmpeg -ss 0.0 -t 294.39079999999996 -i /home/cgn/Downloads/egocom-aligned/vid_